In [ ]:
# %load imports.py
import glob
import os

import h5py
import holoviews as hv
import numpy as np
import pandas as pd
import panel as pn
import param
import yaml
from holoviews import opts
from tqdm import tqdm

hv.extension("bokeh")
from bokeh.io import export_png, export_svgs

opts.defaults(
    opts.Scatter(width=1000, height=300),
    opts.Histogram(width=1000, height=300),
    opts.Image(width=1000, height=300),
    opts.Curve(width=1000, height=300),
    opts.Points(width=1000, height=300),
)


%pylab inline
# from matplotlib.colors import LogNorm
%config InlineBackend.figure_format ='retina'

rcParams["figure.figsize"] = (13.0, 6.0)

from scipy.optimize import curve_fit
from scipy.stats import norm


def get_data_pd(fname: str) -> pd.DataFrame:
    try:
        with h5py.File(fname, "r") as f:
            rawNr = f["raw/trigger nr"][:]
            rawTof = f["raw/tof"][:] * 1e6
            rawTot = f["raw/tot"][:]
            rawX = f["raw/x"][:]
            rawY = f["raw/y"][:]
            centNr = f["centroided/trigger nr"][:]
            centTof = f["centroided/tof"][:] * 1e6
            centTot = f["centroided/tot max"][:]
            centY = f["centroided/y"][:]
            centX = f["centroided/x"][:]

        raw_data = pd.DataFrame(
            np.column_stack((rawNr, rawTof, rawTot, rawX, rawY)),
            columns=("nr", "tof", "tot", "x", "y"),
        )
        cent_data = pd.DataFrame(
            np.column_stack((centNr, centTof, centTot, centX, centY)),
            columns=("nr", "tof", "tot", "x", "y"),
        )
        return raw_data, cent_data
    except:
        print(f'key "{keys}" not known or file "{fname}" not existing')


def gauss_fwhm(x, *p):
    A, mu, fwhm = p
    return A * np.exp(-((x - mu) ** 2) / (2.0 * (fwhm ** 2) / (4 * 2 * np.log(2))))


def find_peaks_in_microbunch(
    data: pd.DataFrame, nr_peaks: int = 4, dt: float = 10, offset: float = 0
) -> list:
    """find first peak in micro-bunch"""
    peaks = []
    for i in range(nr_peaks):
        mask = np.logical_and(
            data["tof"] > (offset + i * dt), data["tof"] < (offset + i * dt + 1)
        )
        x_hist, x_edges = np.histogram(data["tof"][mask], bins=1_000)
        x = (x_edges[:-1] + x_edges[1:]) * 0.5
        popt, pcov = curve_fit(
            gauss_fwhm, x, x_hist, p0=[x_hist.max(), x[x_hist.argmax()], 0.05]
        )
        peaks.append(popt[1])
    return peaks


def shift_microbunch_pulses(
    data: pd.DataFrame, nr_peaks: int = 4, dt: float = 10, offset: float = 0
) -> pd.DataFrame:
    """Fold consecutive micro-bunch pulses back to first"""
    peaks = find_peaks_in_microbunch(data, nr_peaks, dt, offset)

    # shift bunches
    for i in range(1, nr_peaks):
        mask = np.logical_and(
            data["tof"] >= offset + i * dt, data["tof"] < offset + (i + 1) * dt
        )
        data["tof"][mask] -= peaks[i] - peaks[0]

    return data


with open("runs.yaml") as f:
    runNrs = yaml.safe_load(f)

In [ ]:
cmap = "Spectral_r"  # color map for velocity maps
import matplotlib.colors as mplc
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import dask
import dask.bag as db
from dask.distributed import Client

dask.config.set({"distributed.dashboard.link": "/user-redirect/proxy/{port}/status"})
client = Client(n_workers=40, threads_per_worker=2)
client

In [ ]:
def compute_covariance(mz: np.array, bins_mz: np.array) -> np.array:
    """compute 2D covariance map for every trigger event and sum them up"""
    # mz = data[...,2]# for Melby: [...,0]*1e6
    N = len(mz)
    # bins_mz = np.linspace(0, 5, 2000)

    # define necessary function for calculation dask
    @dask.delayed
    def calc_2d_hist(trigger_frames):
        h_2b = np.zeros(2 * (len(bins_mz) - 1,))
        for frame in trigger_frames:
            # call this in dash client
            h_1d_j = np.histogram(frame, bins=bins_mz)[0]
            h_2b += h_1d_j[:, None] * h_1d_j[None, :]
        return h_2b

    @dask.delayed
    def add_matrix(a, b):
        return a + b

    # calculate 2D histogram, covariance mape
    output = []
    for i in range(0, len(mz), 1024):
        output.append(calc_2d_hist(mz[i : i + 1024]))
    zs = dask.persist(*output)

    # tree summation
    empty = np.zeros(2 * (len(bins_mz) - 1,))
    L = zs
    while len(L) > 1:
        new_L = []
        for i in range(0, len(L), 2):
            if i + 1 < len(L):
                lazy = add_matrix(L[i], L[i + 1])
            else:
                lazy = add_matrix(L[i], empty)
            new_L.append(lazy)
        L = new_L

    h_2b = dask.compute(L)[0][0]

    return h_2b

# Prepare data

In [ ]:
file = "out/ion-run_0016_20200903-2202.hdf5"
name = os.path.basename(file).rstrip(".hdf5")
data_raw, data_cent = get_data_pd(file)

In [ ]:
peaks = find_peaks_in_microbunch(
    data_cent,
    nr_peaks=runNrs[name]["pulses"],
    dt=1 / runNrs[name]["rep"] * 1e3,
    offset=0.9,
)

In [ ]:
mask = np.logical_and(data_cent["tof"] > 0, data_cent["tof"] < 20)
x_hist, x_edges = np.histogram(data_cent["tof"][mask], bins=1_000)
a = hv.Histogram((x_hist, x_edges)).opts(
    title=f'{name}: {runNrs[name]["sample"]}, {runNrs[name]["pulses"]}x{runNrs[name]["rep"]} kHz',
    xlabel="TOF (µs)",
    width=1200,
)
b = [hv.VLine(i).opts(line_width=0.5) for i in peaks]
a * b[0] * b[1] * b[2] * b[3] * b[4]

data_shifted = shift_microbunch_pulses(data_cent, nr_peaks=runNrs[name]["pulses"], dt=1/runNrs[name]["rep"]*1e3, offset=0.9)

In [ ]:
# http://holoviews.org/reference/elements/matplotlib/ErrorBars.html
# https://www.nuomiphp.com/eplan/en/29552.html
peaks = find_peaks_in_microbunch(
    data_cent,
    nr_peaks=runNrs[name]["pulses"],
    dt=1 / runNrs[name]["rep"] * 1e3,
    offset=0.9,
)
p = np.polyfit(range(len(peaks)), peaks, deg=1)
poly1d_fn = np.poly1d(p)
x = range(len(peaks))
y = peaks

errors = np.column_stack((x, y, poly1d_fn(x) - y))
hv.Curve((x, poly1d_fn(x))).opts(height=400) * hv.Scatter(errors).opts(
    xlabel="Peak number", ylabel="Peak pos (µs)", color="red"
) * hv.ErrorBars(errors)

In [ ]:
curve_fit(lambda x, m, t: m * x + t, range(len(peaks)), peaks)

In [ ]:
trigger_nr = data_cent["nr"]

In [ ]:
%%time


def foo(i):
    return np.where(trigger_nr == i)[0]


futures = []
nraxis = np.unique(trigger_nr)  # get valid event Nrs
b = db.from_sequence(nraxis, npartitions=100)
b = b.map(foo)

In [ ]:
%time slices = b.compute()

In [ ]:
# test if number of events in slices is the same as in trigger events
_, trigger_frame_len = np.unique(data_cent["nr"], return_counts=True)
x_hist, x_edges = np.histogram(trigger_frame_len, bins=100)
a = hv.Histogram((x_edges[:-1], x_hist)).opts(
    xlabel="Number of clusters per trigger event"
)
assert (
    np.sum(list(map(len, slices)) - trigger_frame_len) == 0
), "the two arrays do not have the same content"
a

In [ ]:
maxlen = np.max(list(map(len, slices)))
print(f"longest mask: {maxlen}")

In [ ]:
# slice vz~0
# original from CH3I_TOFs.ipynb

from scipy.constants import physical_constants

x_cent, y_cent, radius = 136, 140, 112
# https://www.wolframalpha.com/input/?i=26.5713+mm%2F%2810+mm%2Fus+*+2.2567+us%29 from Benjamin Email
vmi_magnification = 28.6445 / (10 * 2.4275)
c_pixel = 78 / (2 * radius) * 1e-3  # 78mm / (224 pixel) conversion von pixel nach m
# https://confluence.desy.de/display/TPX3BT/2020/12/17/VMI+calibration+with+Simion
tof_offset = 0.926 - 0.12  # arrival time of photon peak

df = data_cent.query("tof >= 3.1 & tof < 3.3")
df["tof"] -= tof_offset
df["x_rel"] = df["x"] - x_cent
df["y_rel"] = df["y"] - y_cent
df["r"] = np.sqrt(df["x_rel"] ** 2 + df["y_rel"] ** 2)
df["theta"] = np.arctan2(
    df["y_rel"], df["x_rel"]
)  # alternatively: np.arctan2(df['y'], df['r'])
df["v_x"] = df["x_rel"] * c_pixel / vmi_magnification / (df["tof"] * 1e-6)  # m/s
df["v_y"] = df["y_rel"] * c_pixel / vmi_magnification / (df["tof"] * 1e-6)  # m/s
df["xy_velocity"] = (
    df["r"] * c_pixel / vmi_magnification / (df["tof"] * 1e-6)
)  # for m/s

x_hist, x_edges = np.histogram(df["tof"], bins=100)
x = 0.5 * (x_edges[1:] + x_edges[:-1])
popt, pcov = curve_fit(
    gauss_fwhm, x, x_hist, p0=[x_hist.max(), x[x_hist.argmax()], 0.20]
)

e = physical_constants["elementary charge"][0]  # C
u = physical_constants["atomic mass constant"][0]
E = 269.5 * 100  # 158.9 * 100  # V/m, from Benjamins Simion simulation
m = 14.0067 * u  # kg
Δt = (df["tof"] - popt[1]) * 1e-6  # convert to s
df["vz"] = Δt * e * E / m

tof_t0 = popt[1] + tof_offset

In [ ]:
Δt = 15_000 * 1e6 * m / e / E  # calculate TOF in µs from vz

In [ ]:
# fit first peak
df = data_cent.query("tof >= 3.1 & tof < 3.3")
x_hist, x_edges = np.histogram(df["tof"], bins=200)
x = 0.5 * (x_edges[1:] + x_edges[:-1])
popt, pcov = curve_fit(
    gauss_fwhm, x, x_hist, p0=[x_hist.max(), x[x_hist.argmax()], 0.20]
)

x = 0.5 * (x_edges[1:] + x_edges[:-1])
b = hv.Curve(
    (x, gauss_fwhm(x, *popt)), label=f"FWHM {1e3*popt[2]:.1f} ns, x₀={popt[1]:.3f} µs"
)
hv.Histogram((x_hist, x_edges)).opts(
    xlabel="TOF (µs)", title="N₂ from 1st micro-bunch"
) * b * hv.VLine(tof_t0 - Δt) * hv.VLine(tof_t0 + Δt)

In [ ]:
# see if calculated offset is correct
df = data_cent.query("tof >= 7.1 & tof < 7.33")
df["tof"] -= peaks[1] - peaks[0]
x_hist, x_edges = np.histogram(df["tof"], bins=200)
x = 0.5 * (x_edges[1:] + x_edges[:-1])
popt, pcov = curve_fit(
    gauss_fwhm, x, x_hist, p0=[x_hist.max(), x[x_hist.argmax()], 0.20]
)

x = 0.5 * (x_edges[1:] + x_edges[:-1])
b = hv.Curve(
    (x, gauss_fwhm(x, *popt)), label=f"FWHM {1e3*popt[2]:.1f} ns, x0={popt[1]:.3f} µs"
)
hv.Histogram((x_hist, x_edges)).opts(
    xlabel="TOF (µs)", title="N₂ from 2nd micro-bunch, backfolded"
) * b

In [ ]:
%%time

offset = 0.9
dt = 1 / runNrs[name]["rep"] * 1e3
nr_peaks = runNrs[name]["pulses"]
peaks = find_peaks_in_microbunch(data_cent, nr_peaks=nr_peaks, dt=dt, offset=offset)

nr_events = len(slices)
# initialize empty data array and fill with data
data = np.zeros((nr_peaks * nr_events, maxlen, 3))


@dask.delayed
def micro_pulse_into_data(peak_idx):
    delta = peaks[peak_idx] - peaks[0]

    data = np.zeros((len(slices), maxlen, 3))
    for i, mask in tqdm(enumerate(slices)):
        mask = mask.tolist()
        # shift bunches for whole spectrum
        mask_peak = np.logical_and(
            data_cent["tof"][mask] >= offset + peak_idx * dt,
            data_cent["tof"][mask] < offset + (peak_idx + 1) * dt,
        )
        # tof_t0 gauss fit x0
        # shift peak
        # mask_peak = np.logical_and(
        #    data_cent["tof"][mask] >= peaks[peak_idx] + tof_t0 - peaks[0] - Δt,
        #    data_cent["tof"][mask] <= peaks[peak_idx] + tof_t0 - peaks[0] + Δt,
        # )
        mask_length = mask_peak.sum()  # summation True to see how long the is
        if mask_length > 0:
            data[i, :mask_length, 0] = data_cent["x"][mask][mask_peak]
            data[i, :mask_length, 1] = data_cent["y"][mask][mask_peak]
            data[i, :mask_length, 2] = data_cent["tof"][mask][mask_peak] - delta
            # data[i, :len(mask), 3] = data_cent['tot'][mask]#tot[mask]

    return data


output = []
for peak_idx in range(0, nr_peaks):
    output.append(micro_pulse_into_data(peak_idx))
data_dask = dask.compute(dask.persist(*output))
data_dask = np.concatenate(data_dask[0])

In [ ]:
hv.Histogram(
    np.histogram(
        data_dask[..., 2][(data_dask[..., 2] > 0) & (data_dask[..., 2] < 15)],
        bins=1_000,
    )
)

In [ ]:
%%time
mz = data_dask[..., 2]  # data[:72007,:,2]
bins_mz = np.linspace(1, 5, 5_000)
h_2b = compute_covariance(mz, bins_mz).copy()

In [ ]:
plt.imshow(h_2b, norm=mplc.LogNorm())
plt.colorbar()

In [ ]:
hv.Image(h_2b, bounds=(bins_mz[0], bins_mz[0], bins_mz[-1], bins_mz[-1])).opts(
    colorbar=False, logz=True, clim=(0.1, None), cmap="afmhot_r", width=600, height=600
)

In [ ]:
h_1d = np.histogram(mz.ravel(), bins=bins_mz)[0] / len(data)
h_2d = h_1d[:, None] * h_1d[None, :]
j1d = np.arange(len(bins_mz) - 1)
jx, jy = np.meshgrid(j1d, j1d, indexing="ij")
h_2d[jx >= jy] = 0
h_2b[jx >= jy] = 0

# subtract correlated from uncorrelated map:
h_cov = h_2b / len(mz) - h_2d

In [ ]:
ext_2b = 2 * [bins_mz[0], bins_mz[-1]]
plt.imshow(
    h_cov.T,
    origin="lower",
    extent=ext_2b,
    interpolation="nearest",
    norm=mplc.LogNorm(),
    cmap="afmhot_r",
)
plt.xlabel("TOF (us)")
plt.ylabel("TOF (us)")
# plt.title(f'raw un-centroided {fname}')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(13, 13))
plt.imshow(
    h_cov.T,
    origin="lower",
    extent=ext_2b,
    interpolation="nearest",
    norm=mplc.LogNorm(),
    vmin=1e-6,
    vmax=1e-0,
    #           norm=mplc.SymLogNorm(linthresh=1e-3), vmin=-10, vmax=10,
    cmap="afmhot_r",
)
# plt.colorbar()
plt.xlabel("TOF (us)")
plt.ylabel("TOF (us)")
# plt.ylim(3.1, 3.3)
# plt.xlim(3.1, 3.3)
plt.tight_layout()
# plt.savefig('images/PIPICO_N2.png')

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(
    h_cov.T,
    origin="lower",
    extent=ext_2b,
    interpolation="nearest",
    norm=mplc.LogNorm(),
    vmin=1e-6,
    vmax=5e-1,
    #           norm=mplc.SymLogNorm(linthresh=1e-3), vmin=-10, vmax=10,
    cmap="afmhot_r",
)
# plt.colorbar()
# plt.xlim(2.2, 3.5)
# plt.ylim(2.2, 3.5)
plt.xlabel("TOF (us)")
plt.ylabel("TOF (us)")
plt.tight_layout()
plt.savefig("images/PIPICO_N2_zoom_average.png")

![https://doi.org/10.1103/PhysRevA.94.013426](https://journals.aps.org/pra/article/10.1103/PhysRevA.94.013426/figures/2/medium)
 
Photoion-photoion coincidence map showing the ion breakup channels of N2 from x-ray-pump–x-ray-probe measurements. The axes are the times of flight (TOFs) of the two ions that are detected in coincidence and a momentum conservation filter has been applied. The color intensity scale is normalized to 6720 maximum counts and the total counts is 49 680.

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(
    h_cov.T,
    origin="lower",
    extent=ext_2b,
    interpolation="nearest",
    norm=mplc.LogNorm(),
    vmin=1e-6,
    vmax=5e-1,
    #           norm=mplc.SymLogNorm(linthresh=1e-3), vmin=-10, vmax=10,
    cmap="afmhot_r",
)
# plt.colorbar()
# plt.xlim(1, 2)
# plt.ylim(3, 4)
plt.xlabel("TOF (us)")
plt.ylabel("TOF (us)")
plt.tight_layout()
plt.savefig("images/PIPICO_N2_zoom_average.png")

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(
    h_cov.T,
    origin="lower",
    extent=ext_2b,
    interpolation="nearest",
    norm=mplc.LogNorm(),
    vmin=1e-6,
    vmax=5e-1,
    #           norm=mplc.SymLogNorm(linthresh=1e-3), vmin=-10, vmax=10,
    cmap="afmhot_r",
)
# plt.colorbar()
plt.xlim(3, 3.5)
plt.ylim(3, 3.5)
plt.xlabel("TOF (us)")
plt.ylabel("TOF (us)")
plt.tight_layout()